# Exp P2: Architecture Universality - VGG11 & SimpleCNN (lr=0.01)

## 目的
P実験でVGG11/SimpleCNNが早期崩壊した原因を検証。
lr=0.1はResNet向けに最適化されていたため、lr=0.01で再実験。

## 実験設計
- **Models**: VGG11, SimpleCNN
- **λ**: [0.15, 0.25, 0.35, 0.45, 0.55, 0.65]
- **η**: 0.4（固定）
- **Seeds**: 0-2（3個）
- **lr**: 0.01（変更点）
- **Total**: 2 × 6 × 3 = **36 runs**

## 推定時間
~36 runs × 9 min ≈ **5.5 時間**

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
from datetime import datetime

EXP_NAME = 'exp_P2_architecture_vgg_cnn'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    TIMESTAMP = SAVE_DIR.split('_')[-2] + '_' + SAVE_DIR.split('_')[-1]
    print(f'🔄 Resuming from: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New experiment: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)

print(f'Experiment: {EXP_NAME}')
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# ===== モデル定義 =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

class SimpleCNN(nn.Module):
    """Simple 4-layer CNN for baseline comparison"""
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

class VGG11(nn.Module):
    """VGG11 for CIFAR-10"""
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # Block 4
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # Block 5
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

def get_model(name):
    if name == 'VGG11':
        return VGG11()
    elif name == 'SimpleCNN':
        return SimpleCNN()
    else:
        raise ValueError(f'Unknown model: {name}')

# Test models
for name in ['VGG11', 'SimpleCNN']:
    m = get_model(name)
    params = sum(p.numel() for p in m.parameters())
    print(f'{name}: {params:,} parameters')

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.01  # ★ 変更点: 0.1 → 0.01
K = 16

MODELS = ['VGG11', 'SimpleCNN']
LAMBDAS = [0.15, 0.25, 0.35, 0.45, 0.55, 0.65]
NOISE_RATE = 0.4
SEEDS = [0, 1, 2]

experiments = []
for model in MODELS:
    for lam in LAMBDAS:
        for seed in SEEDS:
            experiments.append({'model': model, 'lambda': lam, 'seed': seed})

print(f'Models: {MODELS}')
print(f'Lambdas: {LAMBDAS}')
print(f'Learning rate: {LR} (changed from 0.1)')
print(f'Total experiments: {len(experiments)}')
print(f'Estimated time: {len(experiments) * 9 / 60:.1f} hours')

# Save config
config = {
    'experiment': EXP_NAME,
    'timestamp': TIMESTAMP,
    'description': 'VGG11/SimpleCNN with lr=0.01 (re-experiment)',
    'parameters': {
        'models': MODELS,
        'lambdas': LAMBDAS,
        'noise_rate': NOISE_RATE,
        'seeds': SEEDS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K
    },
    'total_runs': len(experiments)
}
with open(f'{SAVE_DIR}/{EXP_NAME}_config.json', 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    labels = np.array(labels)
    n_samples = len(labels)
    n_noisy = int(noise_rate * n_samples)
    noisy_idx = np.random.choice(n_samples, n_noisy, replace=False)
    noisy_labels = labels.copy()
    for idx in noisy_idx:
        choices = [l for l in range(10) if l != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def get_data_loaders(trainset, testset):
    train_loader = DataLoader(IndexedDataset(trainset), batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    return train_loader, test_loader

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning =====
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    best_acc = 0
    cos_history = []
    history = {'epoch': [], 'test_acc': [], 'test_error': []}
    
    for epoch in range(EPOCHS):
        model.train()
        epoch_cos = []
        
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_clean = clean_labels_tensor[indices]
            
            # Structure gradient
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Value gradient
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Mix
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
            
            cos_sim = (g_struct_norm @ g_value_norm).item()
            epoch_cos.append(cos_sim)
            
            g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
            
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        cos_history.append(np.mean(epoch_cos))
        
        if (epoch + 1) % 10 == 0:
            acc = evaluate(model, test_loader)
            best_acc = max(best_acc, acc)
            history['epoch'].append(epoch + 1)
            history['test_acc'].append(acc)
            history['test_error'].append(1 - acc)
    
    final_acc = evaluate(model, test_loader)
    avg_cos = np.mean(cos_history)
    
    return final_acc, max(best_acc, final_acc), avg_cos, history

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

# GPU warmup
warmup_model = SimpleCNN().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('GPU warmed up')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/{EXP_NAME}_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['model'], r['lambda'], r['seed']))
    print(f'Checkpoint loaded: {len(results)} runs completed')

total = len(experiments)

for exp in experiments:
    model_name, lam, seed = exp['model'], exp['lambda'], exp['seed']
    
    if (model_name, lam, seed) in completed:
        continue
    
    run_num = len(completed) + 1
    print(f'\n[{run_num}/{total}] {model_name} λ={lam} seed={seed}')
    
    set_seed(seed)
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    model = get_model(model_name).to(device)
    
    start_time = time.time()
    final_acc, best_acc, avg_cos, history = train_dual_gradient(
        model, train_loader, test_loader, clean_labels, noisy_labels, lam
    )
    elapsed = time.time() - start_time
    
    result = {
        'experiment_id': f'P2-{run_num:03d}',
        'experiment': EXP_NAME,
        'model': model_name,
        'lambda': lam,
        'noise_rate': NOISE_RATE,
        'seed': seed,
        'lr': LR,
        'test_acc': final_acc,
        'test_error': 1 - final_acc,
        'best_acc': best_acc,
        'best_error': 1 - best_acc,
        'avg_cos_struct_value': avg_cos,
        'time_seconds': elapsed,
        'history': history
    }
    results.append(result)
    completed.add((model_name, lam, seed))
    
    status = '✅ EXCELLENT' if best_acc > 0.85 else ('⚠️ COLLAPSE' if best_acc < 0.5 else '')
    print(f'  Error: {1-best_acc:.4f} | cos: {avg_cos:.4f} | Time: {elapsed/60:.1f} min {status}')
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    remaining = total - run_num
    eta = remaining * elapsed / 3600
    print(f'  Progress: {run_num}/{total} | ETA: {eta:.1f} hours')
    
    del model
    torch.cuda.empty_cache()

print('\n' + '='*70)
print('ALL EXPERIMENTS COMPLETED!')
print('='*70)

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/{EXP_NAME}_results.json', 'w') as f:
    json.dump(results, f, indent=2)

results_flat = [{k: v for k, v in r.items() if k != 'history'} for r in results]
df = pd.DataFrame(results_flat)
df.to_csv(f'{SAVE_DIR}/{EXP_NAME}_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}')

In [ ]:
# ===== 結果比較 =====
print('='*70)
print('P2 RESULTS: VGG11 & SimpleCNN with lr=0.01')
print('='*70)

for model_name in MODELS:
    print(f'\n=== {model_name} ===')
    print(f'{"λ":>6} | {"error":>8} | {"std":>6} | {"cos":>8} | state')
    print('-' * 55)
    
    model_df = df[df['model'] == model_name]
    for lam in LAMBDAS:
        subset = model_df[model_df['lambda'] == lam]
        if len(subset) == 0:
            continue
        mean_err = subset['best_error'].mean()
        std_err = subset['best_error'].std()
        mean_cos = subset['avg_cos_struct_value'].mean()
        
        if mean_err < 0.15:
            state = '✅ Ordered'
        elif mean_err < 0.30:
            state = '⚡ Critical'
        else:
            state = '❌ Collapse'
        
        print(f'{lam:>6.2f} | {mean_err:>7.1%} | {std_err:>5.1%} | {mean_cos:>+8.3f} | {state}')

print('\n--- Comparison with P (lr=0.1) ---')
print('P (lr=0.1): VGG11 collapsed at λ=0.45, SimpleCNN never reached Ordered')
print('P2 (lr=0.01): Check if wider Ordered region exists')

In [ ]:
# ===== 可視化 =====
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, model_name in enumerate(MODELS):
    ax = axes[i]
    model_df = df[df['model'] == model_name]
    stats = model_df.groupby('lambda')['best_error'].agg(['mean', 'std'])
    
    ax.errorbar(stats.index, stats['mean'], yerr=stats['std'],
                marker='o', capsize=4, linewidth=2, markersize=8)
    ax.axhline(y=0.15, color='green', linestyle='--', alpha=0.5, label='Ordered threshold')
    ax.axhline(y=0.30, color='red', linestyle='--', alpha=0.5, label='Collapse threshold')
    ax.set_xlabel('λ', fontsize=12)
    ax.set_ylabel('Test Error', fontsize=12)
    ax.set_title(f'{model_name} (lr=0.01)', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{EXP_NAME}_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print('Figure saved!')